### Imports

In [1]:
%matplotlib widget
%cd ../../..
import logging
# logging.basicConfig(level=logging.INFO)
import os
import re

from linetimer import CodeTimer
import nibabel
from nibabel import affines, nifti1
import numpy as np
import pyvista
pyvista.set_jupyter_backend('panel')
import trimesh

from BabelBrain.GPUFunctions.GPUVoxelize import Voxelize
from BabelBrain.BabelDatasetPreps import FixMesh


c:\Users\alanc\miniconda3\envs\bbl_dev\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\alanc\OneDrive\Documents\GitHub\BabelBrain


C:\Users\alanc\AppData\Local\Temp\ipykernel_47508\962469944.py:13: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pyvista.set_jupyter_backend('panel')


### GPU Initialization

In [2]:
# GPUBackend='CUDA'
GPUBackend='OpenCL'
# GPUBackend='Metal'
gpu_name = '3060'

Voxelize.InitVoxelize(gpu_name,GPUBackend=GPUBackend)

NVIDIA GeForce RTX 3060
Selecting device:  NVIDIA GeForce RTX 3060


### Load Input Data

In [3]:
T1W_fname = 'C:/Users/alanc/Documents/NeuroFUS/BabelBrain_Test_Data/SDR_0p55/T1W.nii.gz'
stl_fname = 'C:/Users/alanc/Documents/NeuroFUS/BabelBrain_Test_Data/SDR_0p55/m2m_SDR_0p55/skin.stl'
base_name = os.path.splitext(stl_fname)[0]

# Load data
nifti = nibabel.load(T1W_fname)
data = nifti.get_fdata().astype(np.uint8)
mesh = trimesh.load_mesh(stl_fname)

# Fix mesh if needed
if mesh.body_count != 1:
    print("Mesh needs fixing")
    base_name += '_fixed'
    fname = base_name + '.stl'
    if os.path.exists(fname):
        print("Reloading fixed mesh")
        mesh = trimesh.load_mesh(fname)
    else:
        print("Fixing mesh file")
        mesh = FixMesh(mesh)    
        mesh.export(base_name + '.stl')

Mesh needs fixing
Reloading fixed mesh


### Determine Output Data Resolution

In [4]:
# Set desired spatial step
resolutions = {
    'low_freq_low_ppw': 0.919,    # 200 kHz,   6 PPW
    'low_freq_med_ppw': 0.613,    # 200 kHz,   9 PPW
    'low_freq_high_ppw': 0.459,   # 200 kHz,  12 PPW
    'med_freq_low_ppw': 0.306,    # 600 kHz,   6 PPW
    'med_freq_med_ppw': 0.204,    # 600 kHz,   9 PPW
    'med_freq_high_ppw': 0.153,   # 600 kHz,  12 PPW
    'high_freq_low_ppw': 0.184,   # 1000 kHz,  6 PPW
    'high_freq_med_ppw': 0.123,   # 1000 kHz,  9 PPW
    'high_freq_high_ppw': 0.092,  # 1000 kHz, 12 PPW
}
spatial_step = np.full(3,resolutions['low_freq_low_ppw'])
# spatial_step = np.asarray(nifti.header.get_zooms())/2
spatial_step_text = re.sub("\.","_",str(spatial_step[0]))

# Calculate new affine
zooms = np.asarray(nifti.header.get_zooms())
print(f"Original zooms: {zooms}")
print(f"New zooms: {spatial_step}")
new_x = int(nifti.shape[0]/(spatial_step[0]/zooms[0]))
new_y = int(nifti.shape[1]/(spatial_step[1]/zooms[1]))
new_z = int(nifti.shape[2]/(spatial_step[2]/zooms[2]))
affine_upscaled = affines.rescale_affine(nifti.affine.copy(),
                                         nifti.shape,
                                         spatial_step,
                                         (new_x,new_y,new_z))

Original zooms: [0.9992141  0.99999994 0.9999999 ]
New zooms: [0.919 0.919 0.919]


### Run Voxelization Step

In [5]:
with CodeTimer("GPU Voxelization", unit="s"):
    points_voxelization_gpu=Voxelize.Voxelize(mesh,targetResolution=spatial_step[0], GPUBackend=GPUBackend)
    
with CodeTimer("CPU Voxelization", unit="s"):
    voxelization_truth_fname = base_name + f"_voxelization_CPU_spatial_step_{spatial_step_text}.npy"
    if os.path.exists(voxelization_truth_fname):
        print("Reloading CPU file")
        points_voxelization_cpu = np.load(voxelization_truth_fname)
    else:
        print('Generating CPU file')
        points_voxelization_cpu = mesh.voxelized(spatial_step[0],max_iter=30).fill().points
        print('Saving CPU file')
        np.save(voxelization_truth_fname,points_voxelization_cpu)

gpu_vert_num = len(points_voxelization_gpu[:,0])
cpu_vert_num = len(points_voxelization_cpu[:,0])
if gpu_vert_num != cpu_vert_num:
    print(f"Array sizes don't match: {gpu_vert_num} vs {cpu_vert_num}")
if gpu_vert_num == 0:
    print("Arrays are empty")

GPU Voxelizing # triangles 479106
spatial step and  maximal grid dimensions [0.91673711 0.91840075 0.91850272] 173 235 251


c:\Users\alanc\miniconda3\envs\bbl_dev\lib\site-packages\pyopencl\__init__.py:270: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


totalPoints 5219898
globalcount [5219898       0]
Code block 'GPU Voxelization' took: 1.56403 s
Reloading CPU file
Code block 'CPU Voxelization' took: 0.03116 s
Array sizes don't match: 5219898 vs 5327988


### Plot Data

In [6]:
mesh_step = points_voxelization_cpu.shape[0]//100000
mesh_cpu =  pyvista.PolyData(points_voxelization_cpu[::mesh_step,:])
mesh_gpu =  pyvista.PolyData(points_voxelization_gpu[::mesh_step,:])
plotter = pyvista.Plotter(shape=(1,2))
plotter.subplot(0,0)
plotter.add_mesh(pyvista.wrap(mesh),opacity=0.5)
plotter.add_mesh(mesh_cpu,color='blue',opacity=0.1)
plotter.add_text('CPU Voxelization', position='upper_edge', font_size=18)
plotter.subplot(0,1)
plotter.add_mesh(pyvista.wrap(mesh),opacity=0.5)
plotter.add_mesh(mesh_gpu,color='blue',opacity=0.1)
plotter.add_text('GPU Voxelization', position='upper_edge', font_size=18)
plotter.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'9f92799d-9331-414d-a02c-da6bde13ba92': {'version…

### Check Output Data Matches Truth Data

In [7]:
# Convert voxels back to 3D indices
inds_voxelization_gpu = np.hstack((points_voxelization_gpu,np.ones((points_voxelization_gpu.shape[0],1),dtype=points_voxelization_gpu.dtype))).T
inds_voxelization_cpu = np.hstack((points_voxelization_cpu,np.ones((points_voxelization_cpu.shape[0],1),dtype=points_voxelization_cpu.dtype))).T
ijk_gpu_tmp = np.round(np.linalg.inv(affine_upscaled).dot(inds_voxelization_gpu)).T
ijk_cpu_tmp = np.round(np.linalg.inv(affine_upscaled).dot(inds_voxelization_cpu)).T
ijk_gpu = np.ascontiguousarray(ijk_gpu_tmp[:,:3])
ijk_cpu = np.ascontiguousarray(ijk_cpu_tmp[:,:3])

# Remove duplicates
ijk_gpu_unique = np.unique(ijk_gpu, axis=0,)
ijk_cpu_unique = np.unique(ijk_cpu, axis=0)
print(f"Number of CPU indexes: {ijk_cpu_unique.shape[0]}")
print(f"Number of GPU indexes: {ijk_gpu_unique.shape[0]}")

# Count number of matches
set1 = set(map(tuple, ijk_gpu_unique))
set2 = set(map(tuple, ijk_cpu_unique))
common_coordinates = set1.intersection(set2)
match_count = len(common_coordinates)

# Calculate DICE coefficient
dice_coeff = 2 * match_count / (ijk_gpu_unique.shape[0] + ijk_cpu_unique.shape[0])
print(f"Dice coefficient: {dice_coeff}")

Number of CPU indexes: 5215130
Number of GPU indexes: 5093140
Dice coefficient: 0.96647138656632
